In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import math

IMG_SIZE = 640
TEST_IMAGE_DIR = "test/images"
TEST_LABEL_DIR = "test/labels"

CLASS_LABELS = {
    0: "Bad",
    1: "Good"
}

COLORS = {
    0: (0, 255, 0),   # Bad → hijau
    1: (255, 0, 0),   # Good → biru
    2: (0, 0, 255),
}

KEYPOINT_CONNECTIONS = [
    (0, 1),  
    (1, 2),  
]

models = {
    "v8s": YOLO("pose2/train2/weights/best.pt"),
}

def calculate_angle(a, b, c):
    if None in (a, b, c):
        return None
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def draw_actual(image, label_path):
    if not os.path.exists(label_path):
        return image

    h, w = image.shape[:2]
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cls = int(parts[0])
            bbox = list(map(float, parts[1:5]))
            keypoints = list(map(float, parts[5:]))

            xc, yc, bw, bh = bbox
            x1 = int((xc - bw / 2) * w)
            y1 = int((yc - bh / 2) * h)
            x2 = int((xc + bw / 2) * w)
            y2 = int((yc + bh / 2) * h)

            color = COLORS.get(cls, (255, 255, 0))
            label_text = CLASS_LABELS.get(cls, f"Class {cls}")
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, f"{label_text}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            kps = []
            for i in range(0, len(keypoints), 3):
                x = int(keypoints[i] * w)
                y = int(keypoints[i + 1] * h)
                v = int(keypoints[i + 2])
                if v > 0:
                    kps.append((x, y))
                    cv2.circle(image, (x, y), 3, color, -1)
                else:
                    kps.append(None)

            for i, j in KEYPOINT_CONNECTIONS:
                if i < len(kps) and j < len(kps) and kps[i] and kps[j]:
                    cv2.line(image, kps[i], kps[j], color, 1)

            for i in range(len(kps)):
                if i + 2 < len(kps) and all(kps[j] is not None for j in [i, i+1, i+2]):
                    angle = calculate_angle(kps[i], kps[i+1], kps[i+2])
                    if angle is not None:
                        pos = kps[i+1]
                        cv2.putText(image, f"{int(angle)}°", (pos[0] + 5, pos[1] - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return image

def draw_prediction(image, result, label="Model"):
    if result.boxes is not None:
        for box in result.boxes:
            cls = int(box.cls.cpu().item()) if box.cls is not None else -1
            color = COLORS.get(cls, (255, 255, 255))
            label_text = CLASS_LABELS.get(cls, f"Class {cls}")
            xyxy = box.xyxy.cpu().numpy().astype(int)[0]
            conf = box.conf.cpu().item()

            cv2.rectangle(image, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, 2)
            cv2.putText(image, f"{label_text} {conf:.2f}", (xyxy[0], xyxy[1] - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    if hasattr(result, 'keypoints') and result.keypoints is not None:
        for kps_tensor in result.keypoints.data:
            kps_array = kps_tensor.cpu().numpy()
            keypoints = []
            for x, y, conf in kps_array:
                if conf > 0.5:
                    pt = (int(x), int(y))
                    keypoints.append(pt)
                    cv2.circle(image, pt, 3, (0, 0, 255), -1)
                else:
                    keypoints.append(None)

            for i, j in KEYPOINT_CONNECTIONS:
                if keypoints[i] and keypoints[j]:
                    cv2.line(image, keypoints[i], keypoints[j], (0, 0, 255), 1)

            for i in range(len(keypoints)):
                if i + 2 < len(keypoints) and all(keypoints[j] is not None for j in [i, i+1, i+2]):
                    angle = calculate_angle(keypoints[i], keypoints[i+1], keypoints[i+2])
                    if angle is not None:
                        pos = keypoints[i+1]
                        cv2.putText(image, f"{int(angle)}°", (pos[0] + 5, pos[1] - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    return image

for filename in os.listdir(TEST_IMAGE_DIR):
    if not filename.lower().endswith(('.jpg', '.png')):
        continue

    img_path = os.path.join(TEST_IMAGE_DIR, filename)
    label_path = os.path.join(TEST_LABEL_DIR, filename.rsplit('.', 1)[0] + ".txt")

    image = cv2.imread(img_path)
    if image is None:
        print(f"Gagal membuka: {filename}")
        continue

    h0, w0 = image.shape[:2]
    scale = IMG_SIZE / max(h0, w0)
    resized = cv2.resize(image, (int(w0 * scale), int(h0 * scale)))
    padded = np.full((IMG_SIZE, IMG_SIZE, 3), 114, dtype=np.uint8)
    padded[:resized.shape[0], :resized.shape[1]] = resized
    resized = padded

    actual_img = draw_actual(resized.copy(), label_path)
    cv2.putText(actual_img, "Actual", (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    pred_imgs = []
    for model_name, model in models.items():
        result = model.predict(resized, imgsz=IMG_SIZE, conf=0.3, iou=0.45, verbose=False, device='cpu')[0]
        pred_img = draw_prediction(resized.copy(), result, model_name)
        pred_imgs.append(pred_img)

    all_imgs = [actual_img] + pred_imgs
    combined = np.hstack(all_imgs)
    combined_rgb = cv2.cvtColor(combined, cv2.COLOR_BGR2RGB)

    model_titles = ["Actual"] + [f"Model {k}" for k in models.keys()]
    title_text = " | ".join(model_titles)

    plt.figure(figsize=(25, 8))
    plt.imshow(combined_rgb)
    plt.title(f"{title_text} — {filename}", fontsize=14)
    plt.axis("off")
    plt.show()